# Парсинг данных

В рамках данного проекта необходимо осуществить парсинг данных с веб-сайта. В качества веб-сайта взят habr.com. Необходимо осуществить парсинг лучших постов за все время. Данные для парсинга: 1) параметры - наименование статьи, дата загрузки, категория, к которой пост относится; количество голосов и 2) целевой показатель - число просмотров. Параметр 'количество голосов' можно также использовать как целевой показатель, поэтому и информация по нему была собрана. Однако в качестве независимого параметра он не подходит, поскольку может сильно зависеть от количества просмотров.

В дальнейшем можно рассмотреть не только лучшие за все время посты, но и увеличить выборку еще б**о**льшим объемом повседневных постов.

Объем выборки составил 997 статей (для 3 не было обнаружено количество просмотров, поэтому от них решено было избавиться. В дополнение, результат работы парсера (база данных) был сохранен в отдельный файл 'posts_data.csv')

<br>**План проекта**:
1. [Импортирование библиотек](#import)
2. [Парсинг данных](#parsing)
3. [Предобработка данных и Feature Engineering](#preprocessing)

<a name='import'></a>
## 1. Импортирование библиотек

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import re
import pandas as pd

'''
from bs4 import BeautifulSoup
import os
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import cycle
'''

'\nfrom bs4 import BeautifulSoup\nimport os\nimport concurrent.futures\nfrom concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor\nfrom itertools import cycle\n'

<a name='parsing'></a>
## 2. Парсинг данных

Для осуществления парсинга воспользуемся библиотекой selenium. Необходимо получить названия статей, дату их публикации, категории, к которым они относятся, количество голосов и количество просмотров. Для этого будем осуществлять парсинг с помощью бразуера Firefox 50 страниц постов с сайта habr.com по количеству голосов.

In [41]:
firefox_opt = Options()
firefox_opt.page_load_strategy = 'eager'
driver = webdriver.Firefox(options=firefox_opt)

habr_url = 'https://habr.com/ru/top/alltime/'

In [42]:
# Функция для получения данных с одной веб-страницы
def get_page_data(page_num):
    # Заход на веб-страницу
    page_url = f'{habr_url}page{page_num}'
    driver.get(page_url)
    driver.implicitly_wait(5)
    
    # Прокрутка до конца страницы
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Наименования постов
    titles = [title.text for title in driver.find_elements(By.CLASS_NAME, 'tm-title__link')]
    
    # Количество просмотров
    view_counts_link = driver.find_elements(By.CLASS_NAME, 'tm-icon-counter__value')
    view_counts = []
    for view_count in view_counts_link:
        if 'M' in view_count.text:
            view_counts.append(float(re.split('M', view_count.text)[0]) * 1000)
        elif 'K' in view_count.text:
            view_counts.append(float(re.split('K', view_count.text)[0]))
    
    # Категории
    categories_link = driver.find_elements(By.CLASS_NAME, 'tm-article-snippet__hubs-container')
    categories = []
    for category in categories_link:
        categories.append([i for i in re.split('\n', category.text) if i != '*'])

    # Дата и время публикации
    dates_link = driver.find_elements(By.CSS_SELECTOR, 'time')
    dates = [date.get_attribute('datetime') for date in dates_link if 'в' in date.text]
    
    # Количество голосов
    votes_link = driver.find_elements(By.CLASS_NAME, 'tm-votes-meter.tm-data-icons__item')
    votes = [int(vote.text.split('+')[1]) for vote in votes_link]
    
    # Объединение собранных данных в датафрейм
    df = pd.concat(
        [pd.Series(titles), pd.Series(view_counts), pd.Series(categories), pd.Series(dates), pd.Series(votes)],
        axis=1
    )
    df.columns = ['title', 'view_count', 'category', 'date', 'votes']
    
    return df

In [43]:
# Получим данные с каждой страницы топ-50 по постам
df_posts = pd.DataFrame(columns=['title', 'view_count', 'category', 'date'])

for n in range(1, 51):
    df_posts = df_posts.append(get_page_data(n), ignore_index=True)

driver.quit()

In [45]:
# Получим количество пропущенных значений по каждому столбцу
df_posts.isna().sum()

title         0
view_count    3
category      0
date          0
votes         0
dtype: int64

In [46]:
# Избавимся от постов, для которых не было обнаружено 'view_count' (их всего 3) 
df_posts = df_posts[~df_posts['view_count'].isna()].reset_index(drop=True)
df_posts.head()

,title,view_count,category,date,votes
0,Делаем приватный монитор из старого LCD монитора,958.0,[DIY или Сделай сам],2011-11-27T19:21:13.000Z,1448.0
1,Самый беззащитный — уже не Сапсан. Всё оказало...,524.0,"[Информационная безопасность, Системное админи...",2021-01-13T05:51:41.000Z,1447.0
2,Были получены исходники 3300 глобальных интерн...,268.0,[Информационная безопасность],2009-09-23T09:17:27.000Z,1154.0
3,История игрушки. Поле Чудес,290.0,"[История IT, Игры и игровые консоли]",2011-07-18T08:23:11.000Z,913.0
4,"[Обновлено в 10:52, 14.12.19] В офисе Nginx пр...",311.0,"[Блог компании ITSumma, Nginx, Законодательств...",2019-12-12T12:34:52.000Z,791.0


In [47]:
# Сохраним базу данных в отдельный файл 'posts_data.csv'
df_posts.to_csv('posts_data.csv')

<a name='preprocessing'></a>
## 3. Предобработка данных и Feature Enginnering
[Вернуться во Введение](#introduction)

Рассмотрим типы данных каждого из столбца и, в случае необходимости, поменяем их.

In [57]:
df_posts.dtypes

title          object
view_count    float64
category       object
date           object
votes         float64
dtype: object

Можно заметить, что у параметра 'date' тип данных - object, а необходим datatime. Исправим это, поменяв на нужный тип данных. Это позволит создать дополнительные параметры отдельно по году, месяцу, дню, часу и минуте публикации.

In [58]:
df_posts['date'] = pd.to_datetime(df_posts['date'])
df_posts['date'].head()

0   2011-11-27 19:21:13+00:00
1   2021-01-13 05:51:41+00:00
2   2009-09-23 09:17:27+00:00
3   2011-07-18 08:23:11+00:00
4   2019-12-12 12:34:52+00:00
Name: date, dtype: datetime64[ns, UTC]

In [59]:
# Создадим отдельные переменные с годом, месяцем, днем, часом и минутой публикации
df_posts['year'] = df_posts['date'].dt.year
df_posts['month'] = df_posts['date'].dt.month
df_posts['day'] = df_posts['date'].dt.day
df_posts['hour'] = df_posts['date'].dt.hour
df_posts['minute'] = df_posts['date'].dt.minute

df_posts = df_posts.drop('date', axis=1)

In [63]:
df_posts

,title,view_count,category,votes,year,month,day,hour,minute
0,Делаем приватный монитор из старого LCD монитора,958.0,[DIY или Сделай сам],1448.0,2011,11,27,19,21
1,Самый беззащитный — уже не Сапсан. Всё оказало...,524.0,"[Информационная безопасность, Системное админи...",1447.0,2021,1,13,5,51
2,Были получены исходники 3300 глобальных интерн...,268.0,[Информационная безопасность],1154.0,2009,9,23,9,17
3,История игрушки. Поле Чудес,290.0,"[История IT, Игры и игровые консоли]",913.0,2011,7,18,8,23
4,"[Обновлено в 10:52, 14.12.19] В офисе Nginx пр...",311.0,"[Блог компании ITSumma, Nginx, Законодательств...",791.0,2019,12,12,12,34
...,...,...,...,...,...,...,...,...,...
992,Ликбез по типизации в языках программирования,459.0,[Программирование],225.0,2012,12,3,16,46
993,Компьютер-флэшка за $25,28.0,[DIY или Сделай сам],225.0,2011,5,6,9,1
994,Очередная регистрация ООО или жестокая война с...,15.0,[Законодательство в IT],225.0,2011,3,8,21,9
995,По сравнению с тобой большинство людей кажутся...,12.0,[Интерфейсы],225.0,2010,10,26,12,38
